### Limpieza y análisis dataset 3

In [7]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
df3 = pd.read_csv('data/Raw/Study_summary_human.csv')
df3.shape

(5968, 67)

In [9]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5968 entries, 0 to 5967
Data columns (total 67 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   source                                5968 non-null   object 
 1   DOI                                   5968 non-null   object 
 2   Authors                               5968 non-null   object 
 3   Year                                  5968 non-null   int64  
 4   Species                               5968 non-null   object 
 5   Life Stage                            5968 non-null   object 
 6   In vitro/in vivo                      5968 non-null   object 
 7   Sex                                   4383 non-null   object 
 8   Experiment Type                       5968 non-null   object 
 9   Exposure Route                        5968 non-null   object 
 10  Particle Mix?                         5968 non-null   object 
 11  Negative Control 

In [10]:
print("LISTA COMPLETA DE VARIABLES")
for i, columna in enumerate(df3.columns):
    print(f"{i+1:2d}. {columna}")

LISTA COMPLETA DE VARIABLES
 1. source
 2. DOI
 3. Authors
 4. Year
 5. Species
 6. Life Stage
 7. In vitro/in vivo
 8. Sex
 9. Experiment Type
10. Exposure Route
11. Particle Mix?
12. Negative Control
13. Reference Particle
14. Exposure Media
15. Solvent
16. Detergent
17. pH
18. Salinity (ppt)
19. Temperature (Avg)
20. Temperature (Min)
21. Temperature (Max)
22. Exposure Duration (days)
23. Number of Doses
24. Replicates
25. Sample Size
26. Dosing Frequency
27. Nominal Chemical Added
28. Nominal Chemical Added Dose
29. Nominal Chemical Added Dose Units
30. Measured Chemical Added
31. Measured Chemical Added Dose
32. Measured Chemical Added Dose Units
33. Selected Dose
34. Effect
35. Direction
36. Broad Endpoint Category
37. Specific Endpoint Category
38. Endpoint
39. Level of Biological Organization
40. Target Organelle, Cell, or Tissue
41. Polymer
42. Shape
43. Density (g/cm^3)
44. Density, reported or estimated
45. Charge
46. Zeta Potential (mV)
47. Zeta Potential Media
48. Function

In [11]:
# Primera limpieza de columnas que no aportan al modelo
df3 = df3.drop(['source',
                'Authors',
                'Temperature (Min)', 
                'Temperature (Max)', 
                'Nominal Chemical Added', 
                'Nominal Chemical Added Dose',
                'Nominal Chemical Added Dose Units', 
                'Measured Chemical Added',   
                'Measured Chemical Added Dose',   
                'Measured Chemical Added Dose Units', 
                'Direction'], axis = 1)

In [12]:
# Columnas duplicadas
duplicadas = df3.columns[df3.T.duplicated()]
duplicadas

Index([], dtype='object')

In [15]:
df3.duplicated().sum()

np.int64(1386)

In [17]:
df3 = df3.drop_duplicates().reset_index(drop=True)
df3.shape

(4582, 56)

#### Tratamiento de nulos

In [18]:
def inspect_unique_values(df, max_unique=30):

    print("\n============================")
    print(" INSPECCIÓN DE COLUMNAS POR TIPO ")
    print("============================\n")

    for col in df.columns:
        uniques = df[col].dropna().unique()
        num_unique = len(uniques)

        # ¿Es object?
        if df[col].dtype == "object":
            print(f"\n🔵 OBJECT — {col} ({num_unique} categorías)")
            print(sorted(uniques[:max_unique]))
        
        # ¿Es boolean?
        elif df[col].dtype == "bool":
            print(f"\n🟢 BOOLEAN — {col} ({num_unique} valores)")
            print(sorted(uniques))

        # ¿Es numérica con pocas categorías?
        elif pd.api.types.is_numeric_dtype(df[col]) and num_unique < 20:
            print(f"\n🟣 NUMÉRICA-CATEGÓRICA — {col} ({num_unique} valores)")
            print(sorted(uniques))

        # Si no es categórica, no mostramos nada
        else:
            continue

        print("----------------------------------------")


inspect_unique_values(df3)



 INSPECCIÓN DE COLUMNAS POR TIPO 


🔵 OBJECT — DOI (65 categorías)
['10.1002/jbm.a.30354', '10.1002/jbmr.5650080907', '10.1002/pat.3749', '10.1007/s00204-019-02478-7', '10.1007/s00204-020-02750-1', '10.1007/s10904-013-9856-3', '10.1007/s12013-015-0705-6', '10.1016/0142-9612(96)83279-0', '10.1016/S0142-9612(01)00290-3', '10.1016/j.biomaterials.2011.07.037', '10.1016/j.biomaterials.2016.01.064', '10.1016/j.chemosphere.2017.11.076', '10.1016/j.envres.2017.08.043', '10.1016/j.jhazmat.2018.06.017', '10.1016/j.jhazmat.2019.121575', '10.1016/j.scitotenv.2018.08.353', '10.1016/j.scitotenv.2019.05.071', '10.1016/j.scitotenv.2020.138183', '10.1016/j.tiv.2019.104610', '10.1038/srep46687', '10.1039/c7an00461c', '10.1039/c9en00523d', '10.1039/c9tx00147f', '10.1046/j.1525-1594.2003.07107.x', '10.1067/mlc.2001.114677', '10.1080/17435390.2016.1218080', '10.1166/jbn.2006.004', '10.1166/jnn.2019.16347', '10.2147/IJN.S161369', '10.7546/CRABS.2018.10.07']
----------------------------------------

🟣 NUMÉR

In [19]:
# Columnas binarias detectadas que requieren normalización
binary_cols = [
    'Size Validated?',
    'Polymer Validated?',
    'Shape Validated',
    'Sodium Azide Present?',
    'Screened for Chemical Contamination?',
    'Particle Cleaning?',
    'Solvent Rinse',
    'Background Contamination Monitored?',
    'Concentration Validated?',
    'Uptake Validated?',
    'Organisms Fed?',
]

# Diccionario de reemplazo universal
binary_map = {
    'y': 'yes', 'yes': 'yes', 'true': 'yes', '1': 'yes',
    'n': 'no', 'no': 'no', 'false': 'no', '0': 'no',
    'unknown': 'unknown', 'not_reported': 'unknown', 'none': 'unknown',
}

# Limpieza para cada columna
for col in binary_cols:
    if col in df3.columns:

        # Pasar todo a string
        df3[col] = df3[col].astype(str).str.strip().str.lower()

        # Reemplazar valores usando binary_map
        df3[col] = df3[col].replace(binary_map)

        # Los valores que no encajen los convertimos a 'unknown'
        df3[col] = df3[col].where(
            df3[col].isin(['yes', 'no', 'unknown']),
            'unknown'
        )

        # Convertir 'nan' textual a NA real
        df3[col] = df3[col].replace('nan', np.nan)

        print(f"✔ Normalizada: {col}")

print("\nNormalización completada.")

✔ Normalizada: Size Validated?
✔ Normalizada: Polymer Validated?
✔ Normalizada: Shape Validated
✔ Normalizada: Sodium Azide Present?
✔ Normalizada: Screened for Chemical Contamination?
✔ Normalizada: Particle Cleaning?
✔ Normalizada: Solvent Rinse
✔ Normalizada: Background Contamination Monitored?
✔ Normalizada: Concentration Validated?
✔ Normalizada: Uptake Validated?
✔ Normalizada: Organisms Fed?

Normalización completada.
